In [2]:
import os
import pandas as pd
from PIL import Image
import fitz  # PyMuPDF
import pytesseract
import cv2
import numpy as np

# Set up Tesseract path (update to your installation path)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Folder paths
root_folder = r"D:\Selected_Contracts_With_One_BIN"
output_excel_path = r"D:\Selected_Contracts_With_One_BIN_Metadata.xlsx"

# Functions
def get_file_format(file_path):
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    _, ext = os.path.splitext(file_path)
    return ext.lower().strip('.')

def get_tiff_dpi(file_path):
    with Image.open(file_path) as img:
        dpi = img.info.get('dpi', (300, 300))
        return dpi[0]

def get_pdf_dpi(file_path):
    doc = fitz.open(file_path)
    for page in doc:
        pix = page.get_pixmap()
        return pix.xres

def calculate_noise_level(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"File not found or invalid image: {file_path}")
    mean_intensity = np.mean(img)
    std_intensity = np.std(img)
    noise_level = (std_intensity / mean_intensity) * 100
    return noise_level

def is_handwritten(file_path):
    img = cv2.imread(file_path)
    if img is None:
        raise FileNotFoundError(f"File not found or invalid image: {file_path}")
    text_data = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)
    avg_conf = sum(map(int, text_data['conf'])) / len(text_data['conf'])
    return 1 if avg_conf < 50 else 0

def get_tiff_dimensions(file_path):
    with Image.open(file_path) as img:
        return img.size  # (width, height)

def get_pdf_dimensions(file_path):
    doc = fitz.open(file_path)
    dimensions = []
    for page in doc:
        dimensions.append((page.rect.width, page.rect.height))
    return dimensions

# Processing logic
metadata = []

for folder_name in os.listdir(root_folder):
    folder_path = os.path.join(root_folder, folder_name)
    if not os.path.isdir(folder_path):
        continue

    # Find the file (only one PDF or TIFF per folder expected)
    valid_file = None
    for file_name in os.listdir(folder_path):
        if file_name.lower().endswith(('.tif', '.tiff', '.pdf')):
            valid_file = os.path.join(folder_path, file_name)
            break

    if not valid_file:
        print(f"Skipped folder {folder_name}: No valid PDF or TIFF file found.")
        continue

    try:
        file_format = get_file_format(valid_file)
        if file_format in ['tif', 'tiff']:
            resolution = get_tiff_dpi(valid_file)
            dimensions = get_tiff_dimensions(valid_file)
            noise_level = calculate_noise_level(valid_file)
            handwritten = is_handwritten(valid_file)
        elif file_format == 'pdf':
            resolution = get_pdf_dpi(valid_file)
            dimensions = get_pdf_dimensions(valid_file)
            noise_level = "N/A"  # Noise level calculation is for images
            handwritten = "N/A"  # Handwriting detection is for images
        else:
            raise ValueError(f"Unsupported file format: {file_format}")

        metadata.append({
            "Contract Number": folder_name,
            "File Format": file_format,
            "Resolution (DPI)": resolution,
            "Dimensions (Width x Height)": dimensions,
            "Noise Level (%)": noise_level,
            "Is Handwritten": handwritten
        })

        print(f"Processed: {folder_path}")

    except Exception as e:
        print(f"Error processing {folder_path}: {e}")

# Save metadata to Excel
if metadata:
    metadata_df = pd.DataFrame(metadata)
    metadata_df.to_excel(output_excel_path, index=False)
    print(f"Metadata saved to {output_excel_path}")
else:
    print("No valid metadata to save.")


Processed: D:\Selected_Contracts_With_One_BIN\4104.22
Processed: D:\Selected_Contracts_With_One_BIN\5755.65
Processed: D:\Selected_Contracts_With_One_BIN\C236-L46
Processed: D:\Selected_Contracts_With_One_BIN\C310-L57A
Processed: D:\Selected_Contracts_With_One_BIN\C33-14
Processed: D:\Selected_Contracts_With_One_BIN\C34-01
Processed: D:\Selected_Contracts_With_One_BIN\C39-05
Processed: D:\Selected_Contracts_With_One_BIN\C50-02
Processed: D:\Selected_Contracts_With_One_BIN\C52-04
Processed: D:\Selected_Contracts_With_One_BIN\C52-07
Processed: D:\Selected_Contracts_With_One_BIN\C54-10
Processed: D:\Selected_Contracts_With_One_BIN\C54-11
Processed: D:\Selected_Contracts_With_One_BIN\C55-03
Processed: D:\Selected_Contracts_With_One_BIN\C55-10
Processed: D:\Selected_Contracts_With_One_BIN\C55-12
Processed: D:\Selected_Contracts_With_One_BIN\C56-05
Processed: D:\Selected_Contracts_With_One_BIN\C57-11
Processed: D:\Selected_Contracts_With_One_BIN\C57-18
Processed: D:\Selected_Contracts_With_O